In [ ]:
import random
import math

In [ ]:
def euc_2d(c1, c2):
    return round(math.sqrt((c1[0]-c2[0])**2 + (c1[1]-c2[1])**2))

def cost(permutation, cities):
    '''
    Description:
        Calculate the distance between the cities
    '''
    distance = 0
    for i, c1 in enumerate(permutation):
        c2 = permutation[0] if i == (len(permutation)-1) else permutation[i+1]

        distance += euc_2d(cities[c1], cities[c2])

    return distance

def random_permutation(cities):
    '''
    Description:
        Randomly permuate the city indicies
    '''
    perm = [i for i in range(len(cities))]
    for i,_ in enumerate(perm):
        r = random.randint(0, (len(perm)-1-i))+i
        perm[r], perm[i] = perm[i], perm[r]

    return perm

def stochastic_two_opt(perm):
    '''
    Description:
        Permutates a section of the candidate vector. Keeps the rest the same

    Input:
        perm (list): Candidate vector

    Output:
        (list): The candidate vector with a piece reversed
    '''
    # Create two random indicies
    c1, c2 = random.randint(0, (len(perm)-1)), random.randint(0, (len(perm)-1))
    exclude = [c1]

    # Create an Exclude list. This is to ensure that the slice the is being reversed is more than 3 steps wide.
    if c1 == 0:
        exclude.append((len(perm)-1))
    else:
        exclude.append(c1-1)

    if c1 == (len(perm)-1):
        exclude.append(0)
    else:
        exclude.append(c1+1)

    while c2 in exclude:
        c2 = random.randint(0, (len(perm)-1))

    # Reverse the list between the two indicies
    if c2 < c1: c1, c2 = c2, c1
    perm[c1:c2] = perm[c1:c2][::-1]

    return perm

def create_neighbor(current, cities):
    '''
    Description:
        Create a neighbor that is slightly different fromt he candidate vector

    Input:
        current (dictionary): Current candidate vector
        cities (List): The X and Y corrdinates of all the cities

    Output:
        candidate (dictionary): A candidate vector that is slightly different from the original.
    '''
    candidate = {'vector': [], 'cost': None}
    candidate['vector'] = current['vector'].copy()
    stochastic_two_opt(candidate['vector'])
    candidate['cost'] = cost(candidate['vector'], cities)

    return candidate

def should_accept(candidate, current, temp):
    '''
    Description:
        Should the new candidat vector be accepted or not. This is detemined by looking at the
        cost vs the current candidate, or looking at the "temperature" of the sytsem. As
        the algotihm completes iterations, the "temperature cools". This means that the
        algorithm is less liekly to choose a new candidate the longer the algorithm runs

    Inuput:
        candidate (dictionary): New candidate
        current (dictionary): Current best solution
        temp (float): Temperature of the system

    Output:
        (Bool): Should the new candidate be accepted or not
    '''
    if candidate['cost'] <= current['cost']: return True
    return math.exp((current['cost'] - candidate['cost'])/temp) > random.random()

def search(cities, max_iter, max_temp, temp_change):
    '''
    Description:
        Handles all of the function opimization

    Input:
        cities (list): X and Y coordinates of all the cities
        max_iter (int): Number of iterations you want the algorithm to run
        max_temp (float): Starting temperature for the system.
        temp_change (float): The rate as to which the temperature falls with each iteration.
    '''
    current = {'vector': [], 'cost': None}

    current['vector'] = random_permutation(cities)
    current['cost'] = cost(current['vector'], cities)
    temp, best = max_temp, current.copy()

    for i in range(max_iter):
        candidate = create_neighbor(current, cities)
        temp = temp*temp_change

        if should_accept(candidate, current, temp): current = candidate.copy()
        if candidate['cost'] < best['cost']: best = candidate.copy()

        if i%10 == 0:
            print('Iteration: {0}; Best: {1}'.format(i, best['cost']))

    return best

In [ ]:
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]

max_iterations = 2000
max_temp = 100000.0
temp_change = 0.98

best = search(berlin52, max_iterations, max_temp, temp_change)

print('******** Done ********')
print("Best Solution is: Cost: {0}; Vector: {1}".format(best['vector'], best['cost']))